My best stab at a variational autoencoder--the specification of the layers is correct--complete with the nitpicky and unquashable bug at the end which stopped it from ultimately running:

In [109]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
os.environ['KERAS BACKEND'] = 'tensorflow'

from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.losses import binary_crossentropy
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import tensorflow as tf

In [110]:
data = pd.read_parquet('/home/ec2-user/Phys18Conf_1/data_files/final.parquet')
pre_split = data.drop(columns='time')
x_train = pre_split.sample(frac=0.7, random_state=333, axis=1)
x_test = pre_split.drop(columns=x_train.columns)

In [98]:
batch_size = 50
intermediate_dim = 100
original_dim = 14
epochs = 30
latent_dim = 5

In [113]:
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim))
    return z_mean + K.exp(z_log_sigma) * epsilon

x = Input(shape=(original_dim,), name='encoder_input')
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim, name='z_mean')(h)
z_log_sigma = Dense(latent_dim, name='z_log_sigma')(h)

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

encoder = Model(x, z, name='encoder')
encoder.summary()

decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded = decoder_mean(h_decoded)

decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

vae = Model(x, x_decoded, name='vae_mlp')

def vae_loss(x, x_decoded):
    reconstruction_loss = binary_crossentropy(x, x_decoded)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return reconstruction_loss + kl_loss

vae.compile(optimizer='adam', loss=vae_loss)
vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 14)           0                                            
__________________________________________________________________________________________________
dense_96 (Dense)                (None, 100)          1500        encoder_input[0][0]              
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 5)            505         dense_96[0][0]                   
__________________________________________________________________________________________________
z_log_sigma (Dense)             (None, 5)            505         dense_96[0][0]                   
__________________________________________________________________________________________________
lambda_38 

In [114]:
hist = vae.fit(x_train.values.T, x_train.values.T,
               shuffle=True,
               epochs=epochs,
               batch_size=batch_size,
               validation_data=[x_test.values.T, x_test.values.T])

Train on 374 samples, validate on 160 samples
Epoch 1/30
 50/374 [===>..........................] - ETA: 9s - loss: 1277642.2500

InvalidArgumentError: Incompatible shapes: [50,14] vs. [24,14]
	 [[{{node training_20/Adam/gradients/loss_21/dense_98_loss/logistic_loss/mul_grad/BroadcastGradientArgs}} = BroadcastGradientArgs[T=DT_INT32, _class=["loc:@train...ad/Reshape"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](training_20/Adam/gradients/loss_21/dense_98_loss/logistic_loss/mul_grad/Shape, training_20/Adam/gradients/loss_21/dense_98_loss/logistic_loss/mul_grad/Shape_1)]]